In [1]:
import argparse
import time
import numpy as np
import scipy.sparse as sp
import torch
from torch import optim
from model import GCNModelAE, Regularizer
from optimizer import loss_function1
from utils import load_data, preprocess_graph, get_roc_score, load_data_with_labels
from sklearn.cluster import KMeans
from metrics import clustering_metrics

# Hyper-parameter Settings

Here in node clustering we only use half of the training iterations for link prediction (i.e. 100 epochs for Cora and Citeseer, and 750 epochs for PubMed).

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=0, help='Random seed.')
parser.add_argument('--epochs', type=int, default=100, help='Number of epochs to train.')
parser.add_argument('--hidden1', type=int, default=32, help='Number of units in the first encoding layer.')
parser.add_argument('--hidden2', type=int, default=16, help='Number of units in the second embedding layer.')
parser.add_argument('--hidden3', type=int, default=16, help='Number of units in the first hidden layer of Regularizer.')
parser.add_argument('--hidden4', type=int, default=64, help='Number of units in the second hidden layer of Regularizer.')
parser.add_argument('--clamp', type=float, default=0.01, help='Weight clamp for Regularizer Parameters.')
parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate for Generator.')
parser.add_argument('--reglr', type=float, default=0.001, help='Initial learning rate for Regularizer.')
parser.add_argument('--dropout', type=float, default=0., help='Dropout rate (1 - keep probability).')
parser.add_argument('--dataset-str', type=str, default='cora', help='type of dataset.')

args,unknown = parser.parse_known_args()

torch.manual_seed(args.seed)
np.random.seed(args.seed)    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Model for Node Clustering

In [3]:
def gae_for(args):
    print("Using {} dataset".format(args.dataset_str))
    adj, features,true_labels = load_data_with_labels(args.dataset_str)
    n_nodes, feat_dim = features.shape
    features = features.to(device)
    
    if args.dataset_str == 'cora':
        n_clusters = 7
    elif args.dataset_str == 'citeseer':
        n_clusters = 6
    else:
        n_clusters = 3

    # Store original adjacency matrix (without diagonal entries) for later
    adj_orig = adj
    adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
    adj_orig.eliminate_zeros()

    # Some preprocessing
    adj_norm = preprocess_graph(adj)
    adj_norm = adj_norm.to(device)
    
    adj_label = adj + sp.eye(adj.shape[0])
    adj_label = torch.FloatTensor(adj_label.toarray())
    adj_label = adj_label.to(device)

    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

    model = GCNModelAE(feat_dim, args.hidden1, args.hidden2, args.dropout).to(device)
    regularizer = Regularizer(args.hidden3, args.hidden2, args.hidden4).to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    regularizer_optimizer = optim.Adam(regularizer.parameters(), lr=args.reglr)
    
    clustering_scores=[]
    for epoch in range(args.epochs):
        t = time.time()
        model.train()
        regularizer.train() 
        
        #Generate embeddings
        predicted_labels_prob, emb = model(features, adj_norm)
        
        #Wasserstein Regularizer
        for i in range(1):
            f_z = regularizer(emb).to(device)
            r = torch.normal(0.0, 1.0, [n_nodes, args.hidden2]).to(device)
            f_r = regularizer(r)          
            reg_loss = - f_r.mean() + f_z.mean() 
            
            regularizer_optimizer.zero_grad()
            reg_loss.backward(retain_graph=True)
            regularizer_optimizer.step()
            
            # weight clamp
            for p in regularizer.parameters():
                p.data.clamp_(-args.clamp, args.clamp)
        
        #GAE Update
        f_z = regularizer(emb)  
        generator_loss = -f_z.mean()
        loss = loss_function1(preds=predicted_labels_prob, labels=adj_label,
                             norm=norm, pos_weight=torch.tensor(pos_weight))
        loss = loss + generator_loss
        
        optimizer.zero_grad()
        loss.backward()
        cur_loss = loss.item()
        optimizer.step()
     
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss))
        print("time=", "{:.5f}".format(time.time() - t))
        
    np_emb = emb.cpu().detach().numpy()
    kmeans = KMeans(n_clusters= n_clusters, random_state=args.seed).fit(np_emb)
    predict_labels = kmeans.predict(np_emb)
    cm = clustering_metrics(true_labels, predict_labels)
    acc, nmi, f1_macro, precision_macro, adjscore = cm.evaluationClusterModelFromLabel()

    clustering_scores.append([acc, nmi, f1_macro, precision_macro, adjscore])
 
    return clustering_scores[-1]

# Run

In [4]:
once = False

if __name__ == '__main__':
    if once == True:
        gae_for(args)
    else:
        clustering_scores = []
        clustering_metrics_names = ['acc', 'nmi', 'f1_macro', 'precision_macro', 'adjscore']
        
        # using 10 different random seeds
        for seed in range(10):
            print('Seed',seed)
            args.seed = seed
            torch.manual_seed(args.seed)
            clustering_score = gae_for(args)
            clustering_scores.append(clustering_score)
        # show the results by mean and std
        clustering_scores = np.asarray(clustering_scores)
        for i in range(len(clustering_scores[0])):
            print(clustering_metrics_names[i],'=',np.mean(clustering_scores[:,i]),', std = ',np.std(clustering_scores[:,i]))

Seed 0
Using cora dataset
Epoch: 0001 train_loss= 0.77257
time= 0.71816
Epoch: 0002 train_loss= 0.76917
time= 0.02294
Epoch: 0003 train_loss= 0.76390
time= 0.03092
Epoch: 0004 train_loss= 0.75801
time= 0.02892
Epoch: 0005 train_loss= 0.75339
time= 0.02693
Epoch: 0006 train_loss= 0.75271
time= 0.03391
Epoch: 0007 train_loss= 0.75705
time= 0.03391
Epoch: 0008 train_loss= 0.76170
time= 0.02892
Epoch: 0009 train_loss= 0.76304
time= 0.02943
Epoch: 0010 train_loss= 0.76217
time= 0.03192
Epoch: 0011 train_loss= 0.76105
time= 0.02693
Epoch: 0012 train_loss= 0.75910
time= 0.02731
Epoch: 0013 train_loss= 0.75681
time= 0.01795
Epoch: 0014 train_loss= 0.75384
time= 0.02194
Epoch: 0015 train_loss= 0.75081
time= 0.02693
Epoch: 0016 train_loss= 0.74726
time= 0.02944
Epoch: 0017 train_loss= 0.74304
time= 0.02693
Epoch: 0018 train_loss= 0.73815
time= 0.03491
Epoch: 0019 train_loss= 0.73270
time= 0.02793
Epoch: 0020 train_loss= 0.72682
time= 0.02793
Epoch: 0021 train_loss= 0.72065
time= 0.02394
Epoch: 0

Epoch: 0077 train_loss= 0.47721
time= 0.02593
Epoch: 0078 train_loss= 0.47657
time= 0.03590
Epoch: 0079 train_loss= 0.47596
time= 0.02593
Epoch: 0080 train_loss= 0.47538
time= 0.02992
Epoch: 0081 train_loss= 0.47483
time= 0.01994
Epoch: 0082 train_loss= 0.47431
time= 0.02094
Epoch: 0083 train_loss= 0.47381
time= 0.03092
Epoch: 0084 train_loss= 0.47332
time= 0.02792
Epoch: 0085 train_loss= 0.47285
time= 0.02793
Epoch: 0086 train_loss= 0.47240
time= 0.02992
Epoch: 0087 train_loss= 0.47196
time= 0.02593
Epoch: 0088 train_loss= 0.47152
time= 0.03391
Epoch: 0089 train_loss= 0.47110
time= 0.02793
Epoch: 0090 train_loss= 0.47069
time= 0.02593
Epoch: 0091 train_loss= 0.47027
time= 0.02934
Epoch: 0092 train_loss= 0.46987
time= 0.02094
Epoch: 0093 train_loss= 0.46946
time= 0.01895
Epoch: 0094 train_loss= 0.46906
time= 0.02992
Epoch: 0095 train_loss= 0.46867
time= 0.02693
Epoch: 0096 train_loss= 0.46828
time= 0.02892
Epoch: 0097 train_loss= 0.46789
time= 0.02693
Epoch: 0098 train_loss= 0.46750
ti

Epoch: 0055 train_loss= 0.50762
time= 0.01795
Epoch: 0056 train_loss= 0.50644
time= 0.02094
Epoch: 0057 train_loss= 0.50527
time= 0.02493
Epoch: 0058 train_loss= 0.50408
time= 0.02892
Epoch: 0059 train_loss= 0.50285
time= 0.02593
Epoch: 0060 train_loss= 0.50158
time= 0.02693
Epoch: 0061 train_loss= 0.50026
time= 0.02892
Epoch: 0062 train_loss= 0.49890
time= 0.02693
Epoch: 0063 train_loss= 0.49750
time= 0.02693
Epoch: 0064 train_loss= 0.49610
time= 0.03092
Epoch: 0065 train_loss= 0.49468
time= 0.02593
Epoch: 0066 train_loss= 0.49328
time= 0.02593
Epoch: 0067 train_loss= 0.49190
time= 0.01795
Epoch: 0068 train_loss= 0.49056
time= 0.01696
Epoch: 0069 train_loss= 0.48927
time= 0.02094
Epoch: 0070 train_loss= 0.48803
time= 0.03291
Epoch: 0071 train_loss= 0.48685
time= 0.02892
Epoch: 0072 train_loss= 0.48573
time= 0.03191
Epoch: 0073 train_loss= 0.48467
time= 0.02792
Epoch: 0074 train_loss= 0.48366
time= 0.02992
Epoch: 0075 train_loss= 0.48269
time= 0.02394
Epoch: 0076 train_loss= 0.48175
ti

Epoch: 0024 train_loss= 0.63450
time= 0.01995
Epoch: 0025 train_loss= 0.62675
time= 0.02693
Epoch: 0026 train_loss= 0.61881
time= 0.02693
Epoch: 0027 train_loss= 0.61071
time= 0.02843
Epoch: 0028 train_loss= 0.60249
time= 0.02793
Epoch: 0029 train_loss= 0.59419
time= 0.02792
Epoch: 0030 train_loss= 0.58587
time= 0.02693
Epoch: 0031 train_loss= 0.57760
time= 0.02693
Epoch: 0032 train_loss= 0.56946
time= 0.02593
Epoch: 0033 train_loss= 0.56153
time= 0.02793
Epoch: 0034 train_loss= 0.55392
time= 0.01795
Epoch: 0035 train_loss= 0.54670
time= 0.01596
Epoch: 0036 train_loss= 0.53997
time= 0.01795
Epoch: 0037 train_loss= 0.53377
time= 0.01596
Epoch: 0038 train_loss= 0.52813
time= 0.01596
Epoch: 0039 train_loss= 0.52307
time= 0.01795
Epoch: 0040 train_loss= 0.51857
time= 0.01696
Epoch: 0041 train_loss= 0.51462
time= 0.01596
Epoch: 0042 train_loss= 0.51113
time= 0.01995
Epoch: 0043 train_loss= 0.50798
time= 0.01596
Epoch: 0044 train_loss= 0.50508
time= 0.01695
Epoch: 0045 train_loss= 0.50235
ti

ACC=0.721196, f1_macro=0.702950, precision_macro=0.693373, recall_macro=0.734620, f1_micro=0.721196, precision_micro=0.721196, recall_micro=0.721196, NMI=0.536972, ADJ_RAND_SCORE=0.495869
Seed 7
Using cora dataset
Epoch: 0001 train_loss= 0.76767
time= 0.04089
Epoch: 0002 train_loss= 0.76136
time= 0.03092
Epoch: 0003 train_loss= 0.75336
time= 0.02892
Epoch: 0004 train_loss= 0.74451
time= 0.02881
Epoch: 0005 train_loss= 0.73575
time= 0.02992
Epoch: 0006 train_loss= 0.72891
time= 0.02594
Epoch: 0007 train_loss= 0.72577
time= 0.01796
Epoch: 0008 train_loss= 0.72592
time= 0.01695
Epoch: 0009 train_loss= 0.72545
time= 0.02693
Epoch: 0010 train_loss= 0.72222
time= 0.02892
Epoch: 0011 train_loss= 0.71702
time= 0.02892
Epoch: 0012 train_loss= 0.71191
time= 0.02593
Epoch: 0013 train_loss= 0.70632
time= 0.02992
Epoch: 0014 train_loss= 0.70173
time= 0.02793
Epoch: 0015 train_loss= 0.69825
time= 0.02593
Epoch: 0016 train_loss= 0.69453
time= 0.02992
Epoch: 0017 train_loss= 0.69042
time= 0.02992
Epoc

Epoch: 0074 train_loss= 0.46999
time= 0.01596
Epoch: 0075 train_loss= 0.46905
time= 0.01895
Epoch: 0076 train_loss= 0.46814
time= 0.02992
Epoch: 0077 train_loss= 0.46724
time= 0.02992
Epoch: 0078 train_loss= 0.46637
time= 0.02693
Epoch: 0079 train_loss= 0.46553
time= 0.02793
Epoch: 0080 train_loss= 0.46472
time= 0.02793
Epoch: 0081 train_loss= 0.46394
time= 0.02593
Epoch: 0082 train_loss= 0.46320
time= 0.02892
Epoch: 0083 train_loss= 0.46249
time= 0.02593
Epoch: 0084 train_loss= 0.46180
time= 0.02793
Epoch: 0085 train_loss= 0.46114
time= 0.02891
Epoch: 0086 train_loss= 0.46050
time= 0.01995
Epoch: 0087 train_loss= 0.45988
time= 0.01795
Epoch: 0088 train_loss= 0.45928
time= 0.02943
Epoch: 0089 train_loss= 0.45870
time= 0.02593
Epoch: 0090 train_loss= 0.45813
time= 0.02892
Epoch: 0091 train_loss= 0.45759
time= 0.02892
Epoch: 0092 train_loss= 0.45707
time= 0.02693
Epoch: 0093 train_loss= 0.45656
time= 0.02593
Epoch: 0094 train_loss= 0.45608
time= 0.02693
Epoch: 0095 train_loss= 0.45561
ti